# reNews

<br>

### Imports

In [1]:
import pandas as pd

import requests
import xmltodict
from bs4 import BeautifulSoup

from IPython.core.display import JSON

In [3]:
xml['rss']['channel']['item'][0]

OrderedDict([('title', 'Enel turns sod on 163MW Chile solar'),
             ('link',
              'https://renews.biz/64493/enel-turns-sod-on-163mw-chile-solar/'),
             ('pubDate', 'Tue, 17 Nov 2020 09:32:10 GMT'),
             ('dc:creator', 'Sara'),
             ('category', ['Solar', 'Enel Green Power']),
             ('guid',
              'https://renews.biz/64493/enel-turns-sod-on-163mw-chile-solar/'),
             ('description', 'Enel turns sod on 163MW Chile solar')])

In [6]:
list(pd
                    .DataFrame(xml['rss']['channel']['item'])
                    .pipe(lambda df: df.assign(category=df['category'].str.join(', ')))
                    .pipe(lambda df: df.assign(pubDate=pd.to_datetime(df['pubDate']).dt.strftime('%Y-%m-%d %H:%M')))
                    .rename(columns={
                        'link': 'article_url',
                        'description': 'lead',
                        'pubDate': 'date',
                    })
                    .T
                    .to_dict()
                    .values()
                   )

[{'title': 'Enel turns sod on 163MW Chile solar',
  'article_url': 'https://renews.biz/64493/enel-turns-sod-on-163mw-chile-solar/',
  'date': '2020-11-17 09:32',
  'dc:creator': 'Sara',
  'category': 'Solar, Enel Green Power',
  'guid': 'https://renews.biz/64493/enel-turns-sod-on-163mw-chile-solar/',
  'lead': 'Enel turns sod on 163MW Chile solar'},
 {'title': 'SSE Renewables signs Arklow grid agreement',
  'article_url': 'https://renews.biz/64492/irish-project-fuses-offshore-wind-and-data-centre-grid/',
  'date': '2020-11-17 09:06',
  'dc:creator': 'Sara',
  'category': 'Offshore Wind, SSE Renewables',
  'guid': 'https://renews.biz/64492/irish-project-fuses-offshore-wind-and-data-centre-grid/',
  'lead': 'SSE Renewables signs Arklow grid agreement'},
 {'title': 'Sif, Smulders bag Dogger Bank A and B foundations gig',
  'article_url': 'https://renews.biz/64491/sif-smulders-bag-dogger-bank-a-and-b-foundations-gig/',
  'date': '2020-11-17 08:43',
  'dc:creator': 'Sara',
  'category': 'Of

In [ ]:
def retrieve_all_current_articles():
    rss_url = 'http://feeds.feedburner.com/Renews-RenewableEnergyNews'

    r = requests.get(rss_url)
    xml = xmltodict.parse(r.content)

    articles = list(pd
                    .DataFrame(xml['rss']['channel']['item'])
                    .pipe(lambda df: df.assign(enclosure=df['enclosure'].apply(lambda x: x['@url'])))
                    .pipe(lambda df: df.assign(pubDate=pd.to_datetime(df['pubDate']).dt.strftime('%Y-%m-%d %H:%M')))
                    .rename(columns={
                        'link': 'article_url',
                        'description': 'lead',
                        'pubDate': 'date',
                        'source': 'section',
                        'enclosure': 'image_url'
                    })
                    .T
                    .to_dict()
                    .values()
                   )
    
    return articles

articles = retrieve_all_current_articles()

JSON(articles)